## Imports

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import os

from dvn.dvn_fcn import DeepVesselNetFCN
from dvn.solver import Solver
from dvn.data_utils import MRAData
from dvn import patchify_unpatchify as pu

torch.set_default_tensor_type('torch.FloatTensor')
# set up default cuda device
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## MRA dataset

In [9]:
test_MRA = MRAData(root_path="./mra/")
volume = test_MRA[0]

model = DeepVesselNetFCN()
model = torch.load("models/deepvesselnet_noisy.model")

model.eval()

patch_size = (1, 84, 84, 84)
stride = (60, 60, 60)

patches = pu.patchify(volume, patch_size, stride)
patch_shape = patches.shape
patches = Variable(patches.view((-1,) + patch_size).cuda().type(torch.cuda.FloatTensor), requires_grad=False)

output = torch.zeros((0,) + patch_size[1:])

batch_size = 8 #user input
for i in range(np.ceil(1.0 * patches.shape[0] / batch_size)):
	output = torch.cat((output, model(patches[batch_size * i:i * batch_size+batch_size]).data.cpu()), 0) # replace model with your network

new_shape = patch_shape

output = pu.unpatchify(output.view(new_shape), stride, input.shape, 1)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.